In [ ]:
import os
import csv
import cv2
import math
import time
import json
import glob
import shutil
import seaborn as sn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm as tqdm
from tabulate import tabulate


In [ ]:
# Input
# TODO 預測的 mask 結果資料夾
pred_all_crown_list = glob.glob('./result/test_pred_raw/*.PNG')
pred_all_crown_list.sort()
assert len(pred_all_crown_list) != 0, 'wrong pred path'
# TODO 要比較的 label 資料夾
label_folder = '../data/label_CEJline_mask_385_512/'
# label_folder = '../data/7class_generate_tooth_mask/'
assert len(glob.glob(label_folder + '/*.PNG')) != 0, 'wrong label path'

In [ ]:
# Output
# TODO 輸出 csv 設定，會覆蓋原始檔案
output_csv_path = "./IOU_csv/CEJ_385.csv"

In [ ]:
def show_pixel_set(img_nparray):
    a = img_nparray
    unique, counts = np.unique(a, return_counts=True)
    return dict(zip(unique, counts))
def count_iou(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    intersection = np.logical_and(label_target, predict_target)
    union = np.logical_or(label_target, predict_target)
    iou_score = np.sum(intersection) / np.sum(union)
#     print('IOU val:',iou_score)
    return iou_score
def count_pixel_acc(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    different = np.logical_xor(label_target, predict_target)
    diff_score = np.sum(different) / np.sum(np.ones_like(label_img_gray))
#     print('IOU val:',iou_score)
    return 1 - diff_score

In [ ]:
record_iou = pd.DataFrame()
for pred_img_path in tqdm(pred_all_crown_list, total = len(pred_all_crown_list)):
    img_name = os.path.basename(pred_img_path)
    pre_img = cv2.imread(pred_img_path, 0)
    label_img = cv2.imread(label_folder + '/' + img_name, 0)
    if not os.path.exists(label_folder + '/' + img_name):
        print('no label image ', label_folder + '/' + img_name)
        continue

    label_img = cv2.resize(label_img, pre_img.shape)
    label_img = np.where(label_img > 128, 255, 0)
    result_iou = count_iou(pre_img, label_img, 255)
    result_PA = count_pixel_acc(pre_img, label_img, 255)
    record_iou = record_iou.append({'image': img_name, 'iou': result_iou, 'PA': result_PA}\
        , ignore_index=True)
record_iou.to_csv(output_csv_path)

In [ ]:
def show_dis(numbers):
    print(numbers.describe())
    plt.hist(numbers, bins=100)
    plt.show()
show_dis(record_iou['PA'])
show_dis(record_iou['iou'])